In [1]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# DAG: Z → X → Y, Z → Y
model = DiscreteBayesianNetwork([('Z', 'X'), ('Z', 'Y'), ('X', 'Y')])

# P(Z)
cpd_Z = TabularCPD(
    variable='Z',
    variable_card=2,
    values=[[0.6], [0.4]]
)

# P(X | Z)
cpd_X = TabularCPD(
    variable='X',
    variable_card=2,
    values=[[0.8, 0.2], # rows are values of X, columns are values of Z (both binary)
            [0.2, 0.8]],
    evidence=['Z'],
    evidence_card=[2]
)

# P(Y | X, Z)
cpd_Y = TabularCPD(
    variable='Y',
    variable_card=2,
    values=[[0.9, 0.6, 0.3, 0.1], # rows are values of Y, columns are values of X×Z (lexicographically: 00, 01, 10, 11)
            [0.1, 0.4, 0.7, 0.9]],
    evidence=['X', 'Z'],
    evidence_card=[2, 2]
)

# Add CPDs
model.add_cpds(cpd_Z, cpd_X, cpd_Y)
assert model.check_model()

# Observational inference
infer = VariableElimination(model)
result = infer.query(variables=['Y'], evidence={'X': 1})
print(result)


/home/matthewbaggins/code/decision-theory/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


+------+----------+
| Y    |   phi(Y) |
+======+==========+
| Y(0) |   0.1545 |
+------+----------+
| Y(1) |   0.8455 |
+------+----------+


In [ ]:
# 🔪 Intervene: do(X = 1)
# Step 1: Copy and modify the model
do_model = DiscreteBayesianNetwork(model.edges())
do_model.remove_edges_from([('Z', 'X')])  # Remove Z->X; results in: Z->Y->X

# Step 2: Use same CPDs for Z and Y
do_model.add_cpds(cpd_Z, cpd_Y)

# Step 3: Replace CPD for X with a deterministic intervention: P(X=1) = 1
do_cpd_X = TabularCPD(
    variable='X', variable_card=2,
    values=[[0.0], [1.0]]  # P(X=0)=0, P(X=1)=1
)

do_model.add_cpds(do_cpd_X)
assert do_model.check_model()

# Step 4: Inference
do_infer = VariableElimination(do_model)
result = do_infer.query(variables=['Y'])  # P(Y | do(X=1))
print(result)


+------+----------+
| Y    |   phi(Y) |
+======+==========+
| Y(0) |   0.2200 |
+------+----------+
| Y(1) |   0.7800 |
+------+----------+
